# Handling Missing Data

The difference between data found in many tutorials and data in the real world is that real-world data is rarely clean and homogeneous.
In particular, many interesting datasets will have some amount of data missing.
To make matters even more complicated, different data sources may indicate missing data in different ways.

In this section, we will discuss some general considerations for missing data, discuss how Pandas chooses to represent it, and demonstrate some built-in Pandas tools for handling missing data in Python.
Here and throughout the book, we'll refer to missing data in general as *null*, *NaN*, or *NA* values.

## Trade-Offs in Missing Data Conventions

There are a number of schemes that have been developed to indicate the presence of missing data in a table or DataFrame.
Generally, they revolve around one of two strategies: using a *mask* that globally indicates missing values, or choosing a *sentinel value* that indicates a missing entry.

In the masking approach, the mask might be an entirely separate Boolean array, or it may involve appropriation of one bit in the data representation to locally indicate the null status of a value.

In the sentinel approach, the sentinel value could be some data-specific convention, such as indicating a missing integer value with -9999 or some rare bit pattern, or it could be a more global convention, such as indicating a missing floating-point value with NaN (Not a Number), a special value which is part of the IEEE floating-point specification.

None of these approaches is without trade-offs: use of a separate mask array requires allocation of an additional Boolean array, which adds overhead in both storage and computation. A sentinel value reduces the range of valid values that can be represented, and may require extra (often non-optimized) logic in CPU and GPU arithmetic. Common special values like NaN are not available for all data types.

As in most cases where no universally optimal choice exists, different languages and systems use different conventions.
For example, the R language uses reserved bit patterns within each data type as sentinel values indicating missing data, while the SciDB system uses an extra byte attached to every cell which indicates a NA state.

## Missing Data in Pandas

The way in which Pandas handles missing values is constrained by its reliance on the NumPy package, which does not have a built-in notion of NA values for non-floating-point data types.

Pandas could have followed R's lead in specifying bit patterns for each individual data type to indicate nullness, but this approach turns out to be rather unwieldy.
While R contains four basic data types, NumPy supports *far* more than this: for example, while R has a single integer type, NumPy supports *fourteen* basic integer types once you account for available precisions, signedness, and endianness of the encoding.
Reserving a specific bit pattern in all available NumPy types would lead to an unwieldy amount of overhead in special-casing various operations for various types, likely even requiring a new fork of the NumPy package. Further, for the smaller data types (such as 8-bit integers), sacrificing a bit to use as a mask will significantly reduce the range of values it can represent.

NumPy does have support for masked arrays – that is, arrays that have a separate Boolean mask array attached for marking data as "good" or "bad."
Pandas could have derived from this, but the overhead in both storage, computation, and code maintenance makes that an unattractive choice.

With these constraints in mind, Pandas chose to use sentinels for missing data, and further chose to use two already-existing Python null values: the special floating-point ``NaN`` value, and the Python ``None`` object.
This choice has some side effects, as we will see, but in practice ends up being a good compromise in most cases of interest.

### ``None``: Pythonic missing data

The first sentinel value used by Pandas is ``None``, a Python singleton object that is often used for missing data in Python code.
Because it is a Python object, ``None`` cannot be used in any arbitrary NumPy/Pandas array, but only in arrays with data type ``'object'`` (i.e., arrays of Python objects):

In [3]:
import numpy as np
import pandas as pd

In [4]:
vals1 = np.array([1, None, 3, 4]) #Nan
vals1

array([1, None, 3, 4], dtype=object)

In [5]:
vals1.dtype

dtype('O')

In [6]:
vals1.dtype.name

'object'

In [7]:
V = np.arange(12).reshape(3,4)
V

array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11]])

In [8]:
V.dtype

dtype('int64')

In [9]:
V[0][3]

3

In [12]:
V[0][3]=None
V

TypeError: TypeError: int() argument must be a string, a bytes-like object or a number, not 'NoneType'

In [13]:
%timeit  # it is Used in Jupyter to Check the time take Process for this Current Cell
try:
    a = "H2"
    print(int(a))
except NameError as ne:
    print(ne)

ValueError: ValueError: empty body on For

This ``dtype=object`` means that the best common type representation NumPy could infer for the contents of the array is that they are Python objects.
While this kind of object array is useful for some purposes, any operations on the data will be done at the Python level, with much more overhead than the typically fast operations seen for arrays with native types:

In [14]:
np.arange(1E6)

array([0.00000e+00, 1.00000e+00, 2.00000e+00, ..., 9.99997e+05,
       9.99998e+05, 9.99999e+05])

In [15]:
for dtype in ['object','int','float','complex']:
    print("dtype =", dtype)
    %timeit np.arange(1E3, dtype=dtype).sum()
    print()

dtype = object
78.4 µs ± 1.18 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)

dtype = int
9.33 µs ± 31.7 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)

dtype = float
9.19 µs ± 107 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)

dtype = complex
10.4 µs ± 84.1 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)



The use of Python objects in an array also means that if you perform aggregations like ``sum()`` or ``min()`` across an array with a ``None`` value, you will generally get an error:

In [16]:
vals1.sum()

TypeError: TypeError: unsupported operand type(s) for +: 'int' and 'NoneType'

In [17]:
vals1.mean()

TypeError: TypeError: unsupported operand type(s) for +: 'int' and 'NoneType'

In [18]:
vals1+10

TypeError: TypeError: unsupported operand type(s) for +: 'NoneType' and 'int'

This reflects the fact that addition between an integer and ``None`` is undefined.

### ``NaN``: Missing numerical data

The other missing data representation, ``NaN`` (acronym for *Not a Number*), is different; it is a special floating-point value recognized by all systems that use the standard IEEE floating-point representation:

In [19]:
vals2 = np.array([1,3, 4,np.nan]) 
vals2.dtype

dtype('float64')

Notice that NumPy chose a native floating-point type for this array: this means that unlike the object array from before, this array supports fast operations pushed into compiled code.
You should be aware that ``NaN`` is a bit like a data virus–it infects any other object it touches.
Regardless of the operation, the result of arithmetic with ``NaN`` will be another ``NaN``:

In [20]:
1 + np.nan

nan

In [21]:
0 *  np.nan

nan

Note that this means that aggregates over the values are well defined (i.e., they don't result in an error) but not always useful:

In [22]:
vals2.sum(), vals2.min(), vals2.max(),vals2.mean()

(nan, nan, nan, nan)

NumPy does provide some special aggregations that will ignore these missing values:

In [23]:
np.nansum(vals2), np.nanmin(vals2), np.nanmax(vals2),np.nanmean(vals2)

(8.0, 1.0, 4.0, 2.6666666666666665)

Keep in mind that ``NaN`` is specifically a floating-point value; there is no equivalent NaN value for integers, strings, or other types.

### NaN and None in Pandas

``NaN`` and ``None`` both have their place, and Pandas is built to handle the two of them nearly interchangeably, converting between them where appropriate:

In [24]:
pd.Series([1, np.nan, 2, None])

0    1.0
1    NaN
2    2.0
3    NaN
dtype: float64

For types that don't have an available sentinel value, Pandas automatically type-casts when NA values are present.
For example, if we set a value in an integer array to ``np.nan``, it will automatically be upcast to a floating-point type to accommodate the NA:

In [25]:
x = pd.Series(range(4), dtype=int)
x

0    0
1    1
2    2
3    3
dtype: int64

In [26]:
x[3] = None
x

0    0.0
1    1.0
2    2.0
3    NaN
dtype: float64

In [27]:
x.add(2)  # it will Not Effect the orginap data

0    2.0
1    3.0
2    4.0
3    NaN
dtype: float64

In [28]:
np.nan_to_num(x)

array([0., 1., 2., 0.])

In [29]:
np.nanmax(x)

2.0

In [30]:
np.nanmin(x)

0.0

In [31]:
np.nanmean(x),np.nanmedian(x),np.nancumsum(x)

(1.0, 1.0, array([0., 1., 3., 3.]))

Notice that in addition to casting the integer array to floating point, Pandas automatically converts the ``None`` to a ``NaN`` value.
(Be aware that there is a proposal to add a native integer NA to Pandas in the future; as of this writing, it has not been included).

While this type of magic may feel a bit hackish compared to the more unified approach to NA values in domain-specific languages like R, the Pandas sentinel/casting approach works quite well in practice and in my experience only rarely causes issues.

The following table lists the upcasting conventions in Pandas when NA values are introduced:

|Typeclass     | Conversion When Storing NAs | NA Sentinel Value      |
|--------------|-----------------------------|------------------------|
| ``floating`` | No change                   | ``np.nan``             |
| ``object``   | No change                   | ``None`` or ``np.nan`` |
| ``integer``  | Cast to ``float64``         | ``np.nan``             |
| ``boolean``  | Cast to ``object``          | ``None`` or ``np.nan`` |

Keep in mind that in Pandas, string data is always stored with an ``object`` dtype.

In [32]:
x = np.array([10,20,30, np.nan,None])
x

array([10, 20, 30, nan, None], dtype=object)

In [33]:
x.min()

TypeError: TypeError: '<=' not supported between instances of 'float' and 'NoneType'

In [34]:
x.dtype

dtype('O')

In [35]:
Data = pd.Series([10,20,30, np.nan,None])
Data

0    10.0
1    20.0
2    30.0
3     NaN
4     NaN
dtype: float64

In [36]:
Data.min()

10.0

## Operating on Null Values

As we have seen, Pandas treats ``None`` and ``NaN`` as essentially interchangeable for indicating missing or null values.
To facilitate this convention, there are several useful methods for detecting, removing, and replacing null values in Pandas data structures.
They are:

- ``isnull() or isna``: Generate a boolean mask indicating missing values
- ``notnull() or notna``: Opposite of ``isnull()``
- ``dropna()``: Return a filtered version of the data
- ``fillna()``: Return a copy of the data with missing values filled or imputed

We will conclude this section with a brief exploration and demonstration of these routines.

### Detecting null values
Pandas data structures have two useful methods for detecting null data: ``isnull()`` and ``notnull()``.
Either one will return a Boolean mask over the data. For example:

In [37]:
data = pd.Series([1, np.nan, 10.5, None])
data

0     1.0
1     NaN
2    10.5
3     NaN
dtype: float64

In [38]:
data.isnull()

0    False
1     True
2    False
3     True
dtype: bool

In [39]:
data.isna()

0    False
1     True
2    False
3     True
dtype: bool

In [40]:
data.notna() # where You don't have a Null Values

0     True
1    False
2     True
3    False
dtype: bool

In [41]:
data.notnull()

0     True
1    False
2     True
3    False
dtype: bool

In [42]:
Df = pd.DataFrame(data,columns=['Data'])
Df

,Data
0,1.0
1,NaN
2,10.5
3,NaN


In [43]:
Df.isnull()

,Data
0,False
1,True
2,False
3,True


In [44]:
Df.isnull().sum()

Data    2
dtype: int64

In [45]:
Df['Age']=np.array([24,23,45,56])
Df

,Data,Age
0,1.0,24
1,NaN,23
2,10.5,45
3,NaN,56


In [50]:
Df['Gender'] = pd.Categorical(['M','F','M','F'])

In [47]:
Df

,Data,Age,Gender
0,1.0,24,M
1,NaN,23,F
2,10.5,45,M
3,NaN,56,F


In [51]:
Df.dtypes

Data       float64
Age          int64
Gender    category
dtype: object

In [52]:
Df.isnull()

,Data,Age,Gender
0,False,False,False
1,True,False,False
2,False,False,False
3,True,False,False


As mentioned in [Data Indexing and Selection](03.02-Data-Indexing-and-Selection.ipynb), Boolean masks can be used directly as a ``Series`` or ``DataFrame`` index:

In [53]:
Df.isnull().sum()

Data      2
Age       0
Gender    0
dtype: int64

In [54]:
Df.notnull()

,Data,Age,Gender
0,True,True,True
1,False,True,True
2,True,True,True
3,False,True,True


In [55]:
data.notnull()

0     True
1    False
2     True
3    False
dtype: bool

In [56]:
data[data.notnull()]

0     1.0
2    10.5
dtype: float64

In [57]:
Df[Df.notnull()] # with Null Values

,Data,Age,Gender
0,1.0,24,M
1,NaN,23,F
2,10.5,45,M
3,NaN,56,F


In [58]:
Df[Df['Data'].notnull()]  # with out Null Values

,Data,Age,Gender
0,1.0,24,M
2,10.5,45,M


The ``isnull()`` and ``notnull()`` methods produce similar Boolean results for ``DataFrame``s.

### Dropping null values

In addition to the masking used before, there are the convenience methods, ``dropna()``
(which removes NA values) and ``fillna()`` (which fills in NA values). For a ``Series``,
the result is straightforward:

In [59]:
data

0     1.0
1     NaN
2    10.5
3     NaN
dtype: float64

In [60]:
data.dropna() # it will Remove all Null Values but it will not effect the original data

0     1.0
2    10.5
dtype: float64

In [61]:
data

0     1.0
1     NaN
2    10.5
3     NaN
dtype: float64

In [62]:
data.fillna(value=2)

0     1.0
1     2.0
2    10.5
3     2.0
dtype: float64

In [63]:
data.fillna(value={1:2.5,3:50})

0     1.0
1     2.5
2    10.5
3    50.0
dtype: float64

In [64]:
mean = np.nanmean(data)
print(mean)

5.75


In [65]:
data.fillna(value=mean)

0     1.00
1     5.75
2    10.50
3     5.75
dtype: float64

In [66]:
data

0     1.0
1     NaN
2    10.5
3     NaN
dtype: float64

In [67]:
fill = np.nanmedian(data)
fill

5.75

In [68]:
data.fillna(value=mean)

0     1.00
1     5.75
2    10.50
3     5.75
dtype: float64

For a ``DataFrame``, there are more options.
Consider the following ``DataFrame``:

In [69]:
df = pd.DataFrame([[1,      np.nan, 2, None],
                   [2,      3,      5, 10  ],
                   [np.nan, 4,      9, 155 ]],columns=['A','B','C','D'])
df

,A,B,C,D
0,1.0,NaN,2,NaN
1,2.0,3.0,5,10.0
2,NaN,4.0,9,155.0


We cannot drop single values from a ``DataFrame``; we can only drop full rows or full columns.
Depending on the application, you might want one or the other, so ``dropna()`` gives a number of options for a ``DataFrame``.

By default, ``dropna()`` will drop all rows in which *any* null value is present:

In [70]:
df.dropna() # by Default you get row axis=0

,A,B,C,D
1,2.0,3.0,5,10.0


In [71]:
df.dropna(axis=1) # we will get col or axis=1

,C
0,2
1,5
2,9


Alternatively, you can drop NA values along a different axis; ``axis=1`` drops all columns containing a null value:

In [72]:
df.dropna(axis='columns')

,C
0,2
1,5
2,9


In [73]:
df.dropna(axis='rows')

,A,B,C,D
1,2.0,3.0,5,10.0


But this drops some good data as well; you might rather be interested in dropping rows or columns with *all* NA values, or a majority of NA values.
This can be specified through the ``how`` or ``thresh`` parameters, which allow fine control of the number of nulls to allow through.

The default is ``how='any'``, such that any row or column (depending on the ``axis`` keyword) containing a null value will be dropped.
You can also specify ``how='all'``, which will only drop rows/columns that are *all* null values:

In [74]:
df['D'] = np.nan
df

,A,B,C,D
0,1.0,NaN,2,NaN
1,2.0,3.0,5,NaN
2,NaN,4.0,9,NaN


In [75]:
df.dropna(axis='columns', how='all')

,A,B,C
0,1.0,NaN,2
1,2.0,3.0,5
2,NaN,4.0,9


In [76]:
df.dropna(axis=1, how='all')

,A,B,C
0,1.0,NaN,2
1,2.0,3.0,5
2,NaN,4.0,9


In [77]:
df.dropna(axis='columns', how='any') # any null value in the null

,C
0,2
1,5
2,9


In [78]:
df.dropna(axis=0, how='any') #row wise

,A,B,C,D


For finer-grained control, the ``thresh`` parameter lets you specify a minimum number of non-null values for the row/column to be kept:

In [79]:
df

,A,B,C,D
0,1.0,NaN,2,NaN
1,2.0,3.0,5,NaN
2,NaN,4.0,9,NaN


In [80]:
df.dropna(axis='rows', thresh=3)

,A,B,C,D
1,2.0,3.0,5,NaN


In [81]:
df.dropna(axis='rows', thresh=2)

,A,B,C,D
0,1.0,NaN,2,NaN
1,2.0,3.0,5,NaN
2,NaN,4.0,9,NaN


In [0]:
df.dropna(axis='columns', thresh=3)

In [0]:
df.dropna(axis='columns', thresh=2)

In [82]:
Df = pd.DataFrame({'Int':[10,20,30,np.nan,50,60],
                  'Float':[20.5,10.5,5.5,2.5,6.5,np.nan],
                   'Complex':[np.nan,2j+3,np.nan,23j+2,5j+2,np.nan]
                  })
Df

In [84]:
Df.isnull()

In [85]:
Df.isna()

In [86]:
Df.notnull()

In [87]:
Df.notna()

In [88]:
Fill = np.nansum(Df['Int']).astype(float)
Fill

170.0

In [89]:
Fill_Missing_Value = Df.fillna(Fill,axis=1)
Fill_Missing_Value

,Int,Float,Complex
0,10.0,20.5,170.0+0.0j
1,20.0,10.5,3.00+2.00j
2,30.0,5.5,170.0+0.0j
3,170.0,2.5,2.0+23.0j
4,50.0,6.5,2.00+5.00j
5,60.0,170.0,170.0+0.0j


In [90]:
for i in Df.columns:
    fill = np.nanmean(Df[i])
    Df[i].replace(np.nan,fill,inplace=True)
print(Df)

    Int  Float              Complex
0  10.0   20.5  2.333333+10.000000j
1  20.0   10.5   3.000000+2.000000j
2  30.0    5.5  2.333333+10.000000j
3  34.0    2.5  2.0+0000023.000000j
4  50.0    6.5   2.000000+5.000000j
5  60.0    9.1  2.333333+10.000000j



Here the first and last row have been dropped, because they contain only two non-null values.

### Filling null values

Sometimes rather than dropping NA values, you'd rather replace them with a valid value.
This value might be a single number like zero, or it might be some sort of imputation or interpolation from the good values.
You could do this in-place using the ``isnull()`` method as a mask, but because it is such a common operation Pandas provides the ``fillna()`` method, which returns a copy of the array with the null values replaced.

Consider the following ``Series``:

In [91]:
data = pd.Series([1, np.nan, 2, None, 3], index=list('abcde'))
data

a    1.0
b    NaN
c    2.0
d    NaN
e    3.0
dtype: float64

We can fill NA entries with a single value, such as zero:

In [92]:
data.fillna(0)

a    1.0
b    0.0
c    2.0
d    0.0
e    3.0
dtype: float64

We can specify a forward-fill to propagate the previous value forward:

In [93]:
# forward-fill
data.fillna(method='ffill')

a    1.0
b    1.0
c    2.0
d    2.0
e    3.0
dtype: float64

Or we can specify a back-fill to propagate the next values backward:

In [0]:
# back-fill
data.fillna(method='bfill')

For ``DataFrame``s, the options are similar, but we can also specify an ``axis`` along which the fills take place:

In [0]:
df

In [0]:
df.fillna(method='ffill', axis=1) # row 

In [0]:
df.fillna(method='bfill', axis=1)

In [0]:
Data = pd.read_csv('data/california_cities.csv')
Data

In [0]:
Data.info()

In [0]:
Data.describe()

In [0]:
Data_Ch = Data.drop(['Unnamed: 0'],axis=1)

In [0]:
Data_Ch.head()

In [0]:
Data_Ch.info()

In [0]:
Data_Ch.columns

In [0]:
Data_Ch.isna().sum()

In [0]:
fill = np.nanmean(Data_Ch['elevation_m'])
fill

In [0]:
Data_Ch['elevation_m']=Data_Ch['elevation_m'].fillna(fill)

In [0]:
Data_Ch

In [0]:
Data_Ch.info()

In [0]:
Data_Ch['elevation_ft']=Data_Ch['elevation_ft'].fillna(np.nanmean(Data_Ch['elevation_ft']))

In [0]:
Data_Ch.info()

Notice that if a previous value is not available during a forward fill, the NA value remains.